# Spatial transcriptomics unveils the in situ cellular and molecular hallmarks of the lung in fatal COVID-19

# Spatial communication events between transcription factors activity and cell type abundances across COVID-19 induced DAD progression

**Author:** Carlos A. Garcia-Prieto

* This notebook explains the downstream analysis of the spatially mapped and deconvoluted Visium ST data with cell2location, focusing on cell-cell communication events beyond ligand-receptor interactions (LRIs), including spatial dependencies between transcription factors (TF) activity and cell type proportions.
* We followed LIANA+ spatially-informed [bivariate metrics](https://liana-py.readthedocs.io/en/latest/notebooks/bivariate.html), [MISTy](https://liana-py.readthedocs.io/en/latest/notebooks/misty.html) and [decoupleR](https://decoupler-py.readthedocs.io/en/latest/notebooks/spatial.html) tutorials.

## Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
import anndata
import pandas as pd
import scanpy as sc
import plotnine as p9
#!pip install "decoupler>=1.4.0"
import decoupler as dc
import liana as li
from liana.method import MistyData, genericMistyData, lrMistyData
import mudata
from mudata import MuData
import numpy as np
#!pip install squidpy
import squidpy as sq
import pandas as pd
pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt 
import matplotlib as mpl
#import scvi
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs
import cell2location
from cell2location.utils import select_slide
import graphcompass as gc
import os
from scipy.cluster.hierarchy import linkage, dendrogram
import seaborn as sns
import scipy.stats as st
import omnipath as op

Created a temporary directory at /tmp/tmp1lmjivvj
Writing /tmp/tmp1lmjivvj/_remote_module_non_scriptable.py


# Read integrated VISIUM ST data with estimated cell type abundances and TF enrichment results

In [2]:
#Set result folders
results_folder = "/mnt/beegfs/cgarcia/Spatial/COVID19/cell2location/HLCA_publication/HLCA/"

#Create directory for LIANA+ results
directory = f'{results_folder}/liana'
# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

liana_folder = f'{results_folder}/liana/'

#Create directory for bivariate metrics results
directory = f'{results_folder}/liana/bivariate'
# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

bivariate_folder = f'{results_folder}/liana/bivariate/'

#Create directory for TF enrichment results with decoupleR
directory = f'{results_folder}/liana/decoupler'
# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

decoupler_folder = f'{results_folder}/liana/decoupler/'

ref_run_name = f'{results_folder}reference_signatures_finest'
run_name = f'{results_folder}cell2location_map_finest'

In [3]:
#Read anndata with TF enrichment results:
adata_file = f"{bivariate_folder}anndataWith005MinimumExpr/ad_vis_post_distrib_finest_discrete_region_NMF_lineage_decoupler_mlm_Paper.h5ad"
adata_full = sc.read_h5ad(adata_file)

In [4]:
#Read anndata with Top10 enriched TFs activity per condition
acts_collectri_full_file = f"{bivariate_folder}anndataWith005MinimumExpr/acts_collectri_full_mlm_Top10_Paper.h5ad"
acts_collectri_full = sc.read_h5ad(acts_collectri_full_file)

In [5]:
#Read cell type composition
comps_full_file = f"{bivariate_folder}anndataWith005MinimumExpr/comps_full_Paper.h5ad"
comps_full = sc.read_h5ad(comps_full_file)

# Compute Cosine similarity of TFs & cellular composition per Visium ST slide
Cell-cell communication (CCC) is a complex multicellular process. Here we study CCC events beyond protein-mediated LRIs, focusing on the study of spatial dependencies between TFs and cell type proportions.

In [6]:
#Create directory to save results
directory = f'{bivariate_folder}TFs_Comps'
# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

In [7]:
#Set directory
bivariate_TFs_folder = f'{bivariate_folder}TFs_Comps/'

### Plot spatial dependencies between selected cell types and TFs activity

<div class="alert alert-info">
<b>Paper Figure!</b>
Figure 4D
</div>

In [8]:
#Function to compute Cosine similarity between cell type proportions and TFs activity
samples = ["L2P","L19P","L11P","HRC5","HRC6","HRC8","HRC10","HRC11","HRC12","HRC13","HRC16","HRC17","L5P","L14P","L24P","L12P","HRC2","HRC4","HRC18","L3C","L14C","L2C","CONTROL2"]

for sample in samples:
    print(sample)
    #Select sample
    adata = select_slide(adata_full, f'{sample}')
    #Compute spatial neighbors
    li.ut.spatial_neighbors(adata, bandwidth=200, cutoff=0.1, kernel='gaussian', set_diag=True)
    #Format anndata
    adata.var = adata.var.rename_axis(None) #axis=1,axis="columns"
    #Rename cell type columns by removing prefix
    adata.obsm['q05_cell_abundance_w_sf'].columns = adata.obsm['q05_cell_abundance_w_sf'].columns.str.lstrip('q05cell_abundance_w_sf_')
    #Extract cellular compositions
    comps = li.ut.obsm_to_adata(adata, 'q05_cell_abundance_w_sf')
    # Get transcription factor enrichment scores
    est = li.ut.obsm_to_adata(adata, 'collectri_mlm_estimate')
    #Select Top 10 enriched TFs per condition
    top_tfs = acts_collectri_full.var.index
    #Create MuData object with TF activities and cell type proportions, and transfer spatial connectivities and other information from the original annData object
    mdata = MuData({"tf":est, "comps":comps})
    mdata.obsp = adata.obsp
    mdata.uns = adata.uns
    mdata.obsm = adata.obsm
    #Write MuData
    mdata_file_linear = f"{bivariate_TFs_folder}{sample}_Paper.h5mu"
    mdata.write(mdata_file_linear)
    #Compute Cosine similarity
    from itertools import product
    interactions = list(product(comps.var.index, top_tfs))
    li.mt.bivar(mdata,
                x_mod="comps",
                y_mod="tf",
                x_transform=sc.pp.scale,
                y_transform=sc.pp.scale,
                function_name="cosine",
                interactions=interactions,
                mask_negatives=False,
                add_categories=True,
                xy_sep="<->",
                n_perms=100
                )
    #Extract "global" scores (by averaging local cosine similarity scores across all spots) to use them as a global summary: Global mean Cosine similarity and sd
    mdata.mod['local_scores'].uns["global_res"].sort_values("global_mean", ascending=False).to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_{sample}_Paper.csv", index=True)
    mdata.mod['local_scores'].uns["global_res"].sort_values("global_sd", ascending=False).to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_sd_{sample}_Paper.csv", index=True)

    #Select specific spatial dependencies for plotting (Paper Figure 4D)
    TF_global = ['Peribronchial fibroblasts<->SMAD3',
     'Peribronchial fibroblasts<->SMAD7',
     'NK cells<->IKZF1',
     'Monocyte-derived Mph<->MYB',
     'Peribronchial fibroblasts<->ZEB2',
     'AT2<->NKX2-1']
    #Plot cosine similarity scores
    with mpl.rc_context({'axes.facecolor':  'white',
                            'figure.figsize': [6, 6]}):
        sc.pl.spatial(mdata.mod['local_scores'], color=TF_global, size=1.4, cmap="viridis", ncols=3, show=False)
        plt.savefig(f"{bivariate_TFs_folder}global_TF_mean_{sample}_Paper.png",dpi=300, format="png",pad_inches=0.2,bbox_inches="tight")
        plt.close()
    #Plot local categories
    with mpl.rc_context({'axes.facecolor':  'white',
                            'figure.figsize': [6, 6]}):
        sc.pl.spatial(mdata.mod['local_scores'], layer='cats', color=TF_global, size=1.4, cmap='coolwarm', ncols=3, show=False)
        plt.savefig(f"{bivariate_TFs_folder}global_TF_categories_{sample}_Paper.png",dpi=300, format="png",pad_inches=0.2,bbox_inches="tight")
        plt.close()
    #Plot permutation p-values
    with mpl.rc_context({'axes.facecolor':  'white',
                            'figure.figsize': [6, 6]}):
        sc.pl.spatial(mdata.mod['local_scores'], layer='pvals', color=TF_global, size=1.4, cmap='magma_r', ncols=3, show=False)
        plt.savefig(f"{bivariate_TFs_folder}global_TF_pvals_{sample}_Paper.png",dpi=300, format="png",pad_inches=0.2,bbox_inches="tight")
        plt.close()

L2P
L19P
L11P
HRC5
HRC6
HRC8
HRC10
HRC11
HRC12
HRC13
HRC16
HRC17
L5P
L14P
L24P
L12P
HRC2
HRC4
HRC18
L3C
L14C
L2C
CONTROL2


# Compute Moran's R of TFs & cellular composition per Visium ST slide

In [9]:
#Function to compute Moran's R between cell type proportions and TFs activity
samples = ["L2P","L19P","L11P","HRC5","HRC6","HRC8","HRC10","HRC11","HRC12","HRC13","HRC16","HRC17","L5P","L14P","L24P","L12P","HRC2","HRC4","HRC18","L3C","L14C","L2C","CONTROL2"]

for sample in samples:
    print(sample)
    #Select sample
    adata = select_slide(adata_full, f'{sample}')
    #Compute spatial neighbors
    li.ut.spatial_neighbors(adata, bandwidth=200, cutoff=0.1, kernel='gaussian', set_diag=True)
    #Format anndata
    adata.var = adata.var.rename_axis(None) #axis=1,axis="columns"
    #Rename cell type columns by removing prefix
    adata.obsm['q05_cell_abundance_w_sf'].columns = adata.obsm['q05_cell_abundance_w_sf'].columns.str.lstrip('q05cell_abundance_w_sf_')
    #Extract cellular compositions
    comps = li.ut.obsm_to_adata(adata, 'q05_cell_abundance_w_sf')
    # Get transcription factor enrichment scores
    est = li.ut.obsm_to_adata(adata, 'collectri_mlm_estimate')
    #Select Top 10 enriched TFs per condition
    top_tfs = acts_collectri_full.var.index
    #Create MuData object with TF activities and cell type proportions, and transfer spatial connectivities and other information from the original annData object
    mdata = MuData({"tf":est, "comps":comps})
    mdata.obsp = adata.obsp
    mdata.uns = adata.uns
    mdata.obsm = adata.obsm
    #Compute Moran's R
    from itertools import product
    interactions = list(product(comps.var.index, top_tfs))
    li.mt.bivar(mdata,
                x_mod="comps",
                y_mod="tf",
                x_transform=sc.pp.scale,
                y_transform=sc.pp.scale,
                function_name="morans",
                interactions=interactions,
                mask_negatives=False,
                add_categories=True,
                xy_sep="<->"
                )
    #Extract global Moran's R as a global summary
    mdata.mod['local_scores'].uns["global_res"].sort_values("global_r", key=abs, ascending=False).to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_moransR_{sample}_Paper.csv", index=True)
    

L2P
L19P
L11P
HRC5
HRC6
HRC8
HRC10
HRC11
HRC12
HRC13
HRC16
HRC17
L5P
L14P
L24P
L12P
HRC2
HRC4
HRC18
L3C
L14C
L2C
CONTROL2


## Read individual Cosine similarity metrics and append scores 

In [10]:
#Function to read  Cosine similiarty metrics and append scores:
samples = ["L2P","L19P","L11P","HRC5","HRC6","HRC8","HRC10","HRC11","HRC12","HRC13","HRC16","HRC17","L5P","L14P","L24P","L12P","HRC2","HRC4","HRC18","L3C","L14C","L2C","CONTROL2"]

appended_data_cosine_metrics = []
appended_data_sd_cosine_metrics = []

for sample in samples:
    print(sample)
    #Read results
    cosine_metrics = pd.read_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_{sample}_Paper.csv", index_col = 0)
    sd_cosine_metrics = pd.read_csv(f"{bivariate_TFs_folder}bivariate_TF_global_sd_{sample}_Paper.csv", index_col = 0)
    #Set sample ID
    cosine_metrics["sample"] = sample
    sd_cosine_metrics["sample"] = sample
    #Append data
    appended_data_cosine_metrics.append(cosine_metrics)
    appended_data_sd_cosine_metrics.append(sd_cosine_metrics)
    
#Transform data
appended_data_cosine_metrics = pd.concat(appended_data_cosine_metrics)
appended_data_sd_cosine_metrics = pd.concat(appended_data_sd_cosine_metrics)

#Write appended results to DataFrame 
appended_data_cosine_metrics.to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_Cosine_appended_Paper.csv", index=True)
appended_data_sd_cosine_metrics.to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_Cosine_sd_appended_Paper.csv", index=True)


L2P
L19P
L11P
HRC5
HRC6
HRC8
HRC10
HRC11
HRC12
HRC13
HRC16
HRC17
L5P
L14P
L24P
L12P
HRC2
HRC4
HRC18
L3C
L14C
L2C
CONTROL2


## Read individual Moran's R metrics and append scores 

In [11]:
#Function to read  Moran's R metrics and append scores:
samples = ["L2P","L19P","L11P","HRC5","HRC6","HRC8","HRC10","HRC11","HRC12","HRC13","HRC16","HRC17","L5P","L14P","L24P","L12P","HRC2","HRC4","HRC18","L3C","L14C","L2C","CONTROL2"]

appended_data_moransr_metrics = []

for sample in samples:
    print(sample)
    #Read results
    moransr_metrics = pd.read_csv(f"{bivariate_TFs_folder}bivariate_TF_global_moransR_{sample}_Paper.csv", index_col = 0)
    #Set sample ID
    moransr_metrics["sample"] = sample
    #Append data
    appended_data_moransr_metrics.append(moransr_metrics)
    
#Transform data
appended_data_moransr_metrics = pd.concat(appended_data_moransr_metrics)

#Write appended results to DataFrame 
appended_data_moransr_metrics.to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_moransR_appended_Paper.csv", index=True)

L2P
L19P
L11P
HRC5
HRC6
HRC8
HRC10
HRC11
HRC12
HRC13
HRC16
HRC17
L5P
L14P
L24P
L12P
HRC2
HRC4
HRC18
L3C
L14C
L2C
CONTROL2


In [12]:
#Read Cosine similarity scores
TF_cosine_metrics = pd.read_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_Cosine_appended_Paper.csv", index_col = 0)

In [13]:
#Read Cosine similarity standard deviation (sd) scores
TF_sd_cosine_metrics = pd.read_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_Cosine_sd_appended_Paper.csv", index_col = 0)

In [14]:
#Read Moran's R results
TF_moransr_metrics = pd.read_csv(f"{bivariate_TFs_folder}bivariate_TF_global_moransR_appended_Paper.csv", index_col = 0)
TF_moransr_metrics = TF_moransr_metrics.sort_values("global_r", key=abs, ascending=False) #sort by score

## Plot Cosine similarity and Moran´s R scores: spatial dependencies between TFs and cellular composition

In [15]:
#Annotate samples
proliferative = ["L2P","L19P","L11P","HRC5","HRC6","HRC8","HRC10","HRC11","HRC12","HRC13","HRC16","HRC17"]
acute = ["L5P","L14P","L24P","L12P","HRC2","HRC4","HRC18"]
controls = ["L3C","L14C","L2C","CONTROL2"]

In [16]:
#Add condition column
TF_cosine_metrics['Condition'] = np.where(TF_cosine_metrics['sample'].isin(acute) == True, 'DAD_Acute', (np.where(TF_cosine_metrics['sample'].isin(proliferative) == True, 'DAD_Proliferative', 'Control')))
TF_moransr_metrics['Condition'] = np.where(TF_moransr_metrics['sample'].isin(acute) == True, 'DAD_Acute', (np.where(TF_moransr_metrics['sample'].isin(proliferative) == True, 'DAD_Proliferative', 'Control')))

In [17]:
#Sort by interaction and condition
TF_cosine_metrics = TF_cosine_metrics.sort_values(["interaction","Condition"], ascending=True)
TF_moransr_metrics = TF_moransr_metrics.sort_values(["interaction","Condition"], ascending=True)

In [18]:
#Plot Cosine and MoransR metrics by condition
TF_cosine_metrics_plot = TF_cosine_metrics.copy()
TF_cosine_metrics_plot = TF_cosine_metrics_plot[["interaction","global_mean","Condition"]]
TF_moransr_metrics_plot = TF_moransr_metrics.copy()
TF_moransr_metrics_plot = TF_moransr_metrics_plot[["interaction","global_r","Condition"]]

In [19]:
#Add cell type and TF columns
TF_cosine_metrics_plot['Celltype'] = TF_cosine_metrics_plot['interaction'].str.split('<->').str[0]
TF_cosine_metrics_plot['TF'] = TF_cosine_metrics_plot['interaction'].str.split('<->').str[1]
TF_moransr_metrics_plot['Celltype'] = TF_moransr_metrics_plot['interaction'].str.split('<->').str[0]
TF_moransr_metrics_plot['TF'] = TF_moransr_metrics_plot['interaction'].str.split('<->').str[1]

In [20]:
#Create directory to save plots across disease progression
directory = f'{bivariate_folder}Facet'
# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

<div class="alert alert-info">
<b>Paper Figure!</b>
Figure Panel S8A and S8B
</div>

In [21]:
#Plot Global mean Cosine similarity across disease progression per celltype

for celltype in TF_cosine_metrics_plot["Celltype"].value_counts().index.sort_values(): 
    print(celltype)
    TF_cosine_metrics_plot_celltype = TF_cosine_metrics_plot[TF_cosine_metrics_plot["Celltype"]==celltype]
    sns.set(rc={"figure.figsize":(9, 6)})
    sns.set_style("whitegrid")
    sns.set_context("poster")
    with plt.rc_context():  # Use this to set figure params like size and dpi
        g=sns.catplot(data=TF_cosine_metrics_plot_celltype, x="Condition", y="global_mean", col="interaction", kind="box" ,col_wrap=8, palette=sns.color_palette(['#ff7f0e','#1f77b4','#2ca02c']))
        g.set_titles("{col_name}")
        g.tick_params(axis='x', labelrotation=90)
        plt.savefig(f"{bivariate_TFs_folder}Facet/Global_mean_Cosine_similarity_TFs_Composition_Facet_{celltype}_Paper.png",dpi=300, format="png",bbox_inches='tight')
        plt.close()

AT0
AT1
AT2
AT2 proliferating
Adventitial fibroblasts
Alveolar Mph CCL3+
Alveolar Mph MT-positive
Alveolar Mph proliferating
Alveolar fibroblasts
Alveolar macrophages
B cells
Basal resting
CD4 T cells
CD8 T cells
Classical monocytes
Club (non-nasal)
DC1
DC2
Deuterosomal
EC aerocyte capillary
EC arterial
EC general capillary
EC venous pulmonary
EC venous systemic
Interstitial Mph perivascular
Lymphatic EC differentiating
Lymphatic EC mature
Mast cells
Mesothelium
Migratory DCs
Monocyte-derived Mph
Multiciliated (non-nasal)
Myofibroblasts
NK cells
Non-classical monocytes
Peribronchial fibroblasts
Pericytes
Plasma cells
Plasmacytoid DCs
SM activated stress response
Smooth muscle
Subpleural fibroblasts
Suprabasal
T cells proliferating
pre-TB secretory


<div class="alert alert-info">
<b>Paper Figure!</b>
Figure Panel S8A and S8B
</div>

In [22]:
#Plot Global Moran's R across disease progression per celltype

for celltype in TF_moransr_metrics_plot["Celltype"].value_counts().index.sort_values(): 
    print(celltype)
    TF_moransr_metrics_plot_celltype = TF_moransr_metrics_plot[TF_moransr_metrics_plot["Celltype"]==celltype]
    sns.set(rc={"figure.figsize":(9, 6)})
    sns.set_style("whitegrid")
    sns.set_context("poster")
    with plt.rc_context():  # Use this to set figure params like size and dpi
        g=sns.catplot(data=TF_moransr_metrics_plot_celltype, x="Condition", y="global_r", col="interaction", kind="box" ,col_wrap=8, palette=sns.color_palette(['#ff7f0e','#1f77b4','#2ca02c']))
        g.set_titles("{col_name}")
        g.tick_params(axis='x', labelrotation=90)
        plt.savefig(f"{bivariate_TFs_folder}Facet/Global_moransR_similarity_TFs_Composition_Facet_{celltype}_Paper.png",dpi=300, format="png",bbox_inches='tight')
        plt.close()

AT0
AT1
AT2
AT2 proliferating
Adventitial fibroblasts
Alveolar Mph CCL3+
Alveolar Mph MT-positive
Alveolar Mph proliferating
Alveolar fibroblasts
Alveolar macrophages
B cells
Basal resting
CD4 T cells
CD8 T cells
Classical monocytes
Club (non-nasal)
DC1
DC2
Deuterosomal
EC aerocyte capillary
EC arterial
EC general capillary
EC venous pulmonary
EC venous systemic
Interstitial Mph perivascular
Lymphatic EC differentiating
Lymphatic EC mature
Mast cells
Mesothelium
Migratory DCs
Monocyte-derived Mph
Multiciliated (non-nasal)
Myofibroblasts
NK cells
Non-classical monocytes
Peribronchial fibroblasts
Pericytes
Plasma cells
Plasmacytoid DCs
SM activated stress response
Smooth muscle
Subpleural fibroblasts
Suprabasal
T cells proliferating
pre-TB secretory


In [23]:
#Create directory to save box plots with scores
directory = f'{bivariate_folder}Global'
# Check if the directory exists
if not os.path.exists(directory):
    # If it doesn't exist, create it
    os.makedirs(directory)

In [24]:
#Order interaction by Global mean Cosine similarity median score
TFcosineorder = TF_cosine_metrics_plot.groupby('interaction')['global_mean'].median().sort_values(ascending = False).index
TFcosine_order = {nm:ix for ix,nm in enumerate(TFcosineorder)} # generated
TF_cosine_metrics_plot_order = TF_cosine_metrics_plot.sort_values(by='interaction', key=lambda nm: nm.map(TFcosine_order))

In [25]:
#Order interaction by Global Moran's R median score
TFmoransrorder = TF_moransr_metrics_plot.groupby('interaction')['global_r'].median().sort_values(ascending = False).index
TFmoransr_order = {nm:ix for ix,nm in enumerate(TFmoransrorder)} # generated
TF_moransr_metrics_plot_order = TF_moransr_metrics_plot.sort_values(by='interaction', key=lambda nm: nm.map(TFmoransr_order))

<div class="alert alert-info">
<b>Paper Figure!</b>
Figure Panel 4C and S8B
</div>

In [26]:
#Plot box plots with Global mean Cosine similarity per celltype

for celltype in TF_cosine_metrics_plot_order["Celltype"].value_counts().index.sort_values(): 
    print(celltype)
    TF_cosine_metrics_plot_global = TF_cosine_metrics_plot_order[TF_cosine_metrics_plot_order["Celltype"]==celltype]
    sns.set(rc={"figure.figsize":(10, 7)})
    sns.set_style("whitegrid")
    with plt.rc_context():  # Use this to set figure params like size and dpi
        ax=sns.boxplot(x='TF', y='global_mean', data=TF_cosine_metrics_plot_global, color='grey')
        ax.tick_params(axis='x', labelrotation=90)
        plt.savefig(f"{bivariate_TFs_folder}Global/Global_mean_Cosine_similarity_TF_{celltype}_order_Paper.png",dpi=300, format="png", bbox_inches='tight')
        plt.close()

AT0
AT1
AT2
AT2 proliferating
Adventitial fibroblasts
Alveolar Mph CCL3+
Alveolar Mph MT-positive
Alveolar Mph proliferating
Alveolar fibroblasts
Alveolar macrophages
B cells
Basal resting
CD4 T cells
CD8 T cells
Classical monocytes
Club (non-nasal)
DC1
DC2
Deuterosomal
EC aerocyte capillary
EC arterial
EC general capillary
EC venous pulmonary
EC venous systemic
Interstitial Mph perivascular
Lymphatic EC differentiating
Lymphatic EC mature
Mast cells
Mesothelium
Migratory DCs
Monocyte-derived Mph
Multiciliated (non-nasal)
Myofibroblasts
NK cells
Non-classical monocytes
Peribronchial fibroblasts
Pericytes
Plasma cells
Plasmacytoid DCs
SM activated stress response
Smooth muscle
Subpleural fibroblasts
Suprabasal
T cells proliferating
pre-TB secretory


<div class="alert alert-info">
<b>Paper Figure!</b>
Figure Panel 4C and S8B
</div>

In [27]:
#Plot box plots with Global Moran's R per celltype

for celltype in TF_moransr_metrics_plot_order["Celltype"].value_counts().index.sort_values(): 
    print(celltype)
    TF_moransr_metrics_plot_global = TF_moransr_metrics_plot_order[TF_moransr_metrics_plot_order["Celltype"]==celltype]
    sns.set(rc={"figure.figsize":(10, 7)})
    sns.set_style("whitegrid")
    with plt.rc_context():  # Use this to set figure params like size and dpi
        ax=sns.boxplot(x='TF', y='global_r', data=TF_moransr_metrics_plot_global, color='grey')
        ax.tick_params(axis='x', labelrotation=90)
        plt.savefig(f"{bivariate_TFs_folder}Global/Global_moransR_TF_{celltype}_order_Paper.png",dpi=300, format="png", bbox_inches='tight')
        plt.close()

AT0
AT1
AT2
AT2 proliferating
Adventitial fibroblasts
Alveolar Mph CCL3+
Alveolar Mph MT-positive
Alveolar Mph proliferating
Alveolar fibroblasts
Alveolar macrophages
B cells
Basal resting
CD4 T cells
CD8 T cells
Classical monocytes
Club (non-nasal)
DC1
DC2
Deuterosomal
EC aerocyte capillary
EC arterial
EC general capillary
EC venous pulmonary
EC venous systemic
Interstitial Mph perivascular
Lymphatic EC differentiating
Lymphatic EC mature
Mast cells
Mesothelium
Migratory DCs
Monocyte-derived Mph
Multiciliated (non-nasal)
Myofibroblasts
NK cells
Non-classical monocytes
Peribronchial fibroblasts
Pericytes
Plasma cells
Plasmacytoid DCs
SM activated stress response
Smooth muscle
Subpleural fibroblasts
Suprabasal
T cells proliferating
pre-TB secretory


### Merge Global mean Cosine similarity and Global Moran´s R scores

In [28]:
#Create key to merge
TF_cosine_metrics["key"] = TF_cosine_metrics["interaction"] + "_" + TF_cosine_metrics["sample"]
TF_moransr_metrics["key"] = TF_moransr_metrics["interaction"] + "_" + TF_moransr_metrics["sample"]

In [29]:
#Merge two dataframes
TF_cosine_moransr_merged = pd.merge(TF_cosine_metrics, TF_moransr_metrics[["global_r","global_pvals","key"]], on='key', how='outer')

In [30]:
#Write merged df
TF_cosine_moransr_merged.to_csv(f"{bivariate_TFs_folder}bivariate_TF_global_mean_Cosine_MoransR_merged_Paper.csv", index=True)

### Modules and their versions used for this analysis


Useful for debugging and reporting issues.

In [31]:
cell2location.utils.list_imported_modules()

sys 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
re 2.2.1
ipykernel._version 6.25.2
json 2.0.9
jupyter_client._version 8.3.1
platform 1.0.8
_ctypes 1.1.0
ctypes 1.1.0
zmq.sugar.version 25.1.1
zmq.sugar 25.1.1
zmq 25.1.1
logging 0.5.1.2
traitlets._version 5.10.0
traitlets 5.10.0
jupyter_core.version 5.3.1
jupyter_core 5.3.1
tornado 6.3.3
zlib 1.0
_curses b'2.2'
socketserver 0.4
argparse 1.1
dateutil._version 2.8.2
dateutil 2.8.2
six 1.16.0
_decimal 1.70
decimal 1.70
platformdirs.version 3.10.0
platformdirs 3.10.0
_csv 1.0
csv 1.0
jupyter_client 8.3.1
ipykernel 6.25.2
IPython.core.release 8.15.0
executing.version 1.2.0
executing 1.2.0
pure_eval.version 0.2.2
pure_eval 0.2.2
stack_data.version 0.6.2
stack_data 0.6.2
pygments 2.16.1
ptyprocess 0.7.0
pexpect 4.8.0
IPython.core.crashhandler 8.15.0
pickleshare 0.7.5
backcall 0.2.0
decorator 5.1.1
_sqlite3 2.6.0
sqlite3.dbapi2 2.6.0
sqlite3 2.6.0
exceptiongroup._version 1.1.3
exceptiongroup 1.1.3
wcwidth 0.2.6
prompt_toolkit 3.0.39
parso